In [1]:
!pip install --upgrade langsmith
!pip install langchain chroma
!pip install chromadb
!pip install langchain_community
!pip install --upgrade langchain_aws

import os

print("Langchain key to start tracing")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "pr-aching-poisoning-52"
os.environ["LANGCHAIN_API_KEY"] = "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"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 90.3 MB/s eta 0:00:00
Langchain key to start tracing


In [2]:

import boto3


AWS_REGION = "us-east-1"

bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name=AWS_REGION,
)

In [3]:
from langchain_aws import BedrockLLM

llm = BedrockLLM(client=bedrock_client,model_id="amazon.titan-text-express-v1")

In [4]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
# bs4_strainer = bs4.SoupStrainer(class_=("the-python-tutorial"))
loader = WebBaseLoader(
    web_paths=("https://www.ibm.com/think/topics/data-science",),
    # bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

len(docs[0].page_content)

USER_AGENT environment variable not set, consider setting it to identify your requests.


18108

In [5]:
docs

[Document(metadata={'source': 'https://www.ibm.com/think/topics/data-science', 'title': 'What is Data Science? | IBM', 'description': 'Data science is a multidisciplinary approach to gaining insights from an increasing amount of data. IBM data science products help find the value of your data.', 'language': 'en'}, page_content='\n\n\n\n\n\n\n\n\nWhat is Data Science? | IBM\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                        \n\n\n\n  \n    What is data science?\n\n\n\n\n\n\n    \n\n\n                    \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                    Analytics\n                                \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                Link copied\n            \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n        What is data science?\r\n    \n\n\n\nData science combines math an

In [6]:
print(docs[0].page_content[:200])










What is Data Science? | IBM













































































                        



  
    What is data science?






    


                


In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=150, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

55

In [8]:
all_splits

[Document(metadata={'source': 'https://www.ibm.com/think/topics/data-science', 'title': 'What is Data Science? | IBM', 'description': 'Data science is a multidisciplinary approach to gaining insights from an increasing amount of data. IBM data science products help find the value of your data.', 'language': 'en', 'start_index': 9}, page_content='What is Data Science? | IBM\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                        \n\n\n\n  \n    What is data science?\n\n\n\n\n\n\n    \n\n\n                    \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                    Analytics\n                                \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                Link copied\n            \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n        What is data science?'),
 Document(metadata={'source': 'https://ww

In [9]:


from langchain.vectorstores import Chroma
from langchain_aws.embeddings.bedrock import BedrockEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=BedrockEmbeddings(client=bedrock_client,model_id="amazon.titan-embed-text-v2:0"))

In [10]:
vectorstore

In [11]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2})

retrieved_docs = retriever.invoke("difference between data science and cloud computing")

len(retrieved_docs)

2

In [12]:
print(retrieved_docs[0].page_content)
print('---------')
print(retrieved_docs[1].page_content)


Data science and cloud computing
---------
Cloud computing scales data science by providing access to additional processing power, storage, and other tools required for data science projects.
